### Quantum Battery project

Step 1. Run below cell to import all package.

In [1]:
from qsee.compilation.qcompilation import QuantumCompilation
from qsee.compilation.qsp import QuantumStatePreparation
from qsee.core import ansatz, state, gradient, optimizer
from qsee.backend import constant, utilities
import matplotlib.pyplot as plt
import base.constant as ct
import base.calculation as ca
import base.hamiltonian as hm
import base.circuit as cr
import matplotlib.pyplot as plt
import numpy as np, qiskit
from qiskit import QuantumCircuit
%load_ext autoreload
%autoreload 2

Step 2. Run below cell to make sure that your environment is ok or not

In [ ]:
qsp = QuantumStatePreparation.prepare([1, 0, 0, 0])

Step 3. Coding!!!

In [ ]:
t = 10
thetas = np.zeros(2*(ct.N-1))
# Calculate W(t) and P(t) by measurement (measure many times) or theory (caculate vector @ matrix @ vector).
ltime,arrayW,arrayP = ca.manytime(t,thetas,10)
plt.plot(ltime,arrayW,arrayP)

